In [8]:
import sys
sys.path.append('../')

from src.m4_generator import M4Generator
from src.m4_data_loader import M4DataLoader
from src.m4_evaluation_loss_functions import *
from src.visualization import *
from src.evaluation import *
from src.m4_model import M4Model
from glob import glob

import numpy as np
import keras
import matplotlib.pyplot as plt

import tensorflow as tf

%matplotlib inline

# Full Evaluation And Comparsion ( Point Prediction)
Evaluate all trained models aginst all error functions

In [3]:

diffAugmentation = DiffAugmentation()
x_augmentations = []
y_augmentations = []

data = {'Model':[], 'Training Loss Function':[], 'Epochs':[], 'LOOKBACK': [], 'Hidden Layer Size':[],
        'Features Number':[], 'T-MASE': [], 'T-MASE Naive': [], 'V-MASE': [], 'V-MASE Naive': [],
        'T-Imporvement %': [],'V-Imporvement %': [], 
       }

models_arch = ['1-LSTM', '2-LSTM', '3-LSTM']
loss_functions = [m4_mase]

MODEL_BASE_DIR = '../models/berken'
TRAINING_DATA_PATH = '../Dataset/Train/Hourly-train.csv'
TEST_DATA_PATH = '../Dataset/Test/Hourly-test.csv'

for model_arc in models_arch:
    s = f'{MODEL_BASE_DIR}/{model_arc}/*/'
    models = glob(s)
    for model_path in models:
        
        ev_results = {}
        
        for loss in loss_functions:
            ev_results = load_and_evaluate_model(model_path, '../Dataset/Train/Hourly-train.csv', 
                                                 '../Dataset/Test/Hourly-test.csv', 
                                                 x_augmentations, y_augmentations, loss)
        data['Model'].append(model_arc)
        data['Training Loss Function'].append(ev_results['hyperparameters']['loss'])
        data['Epochs'].append(ev_results['hyperparameters']['epochs'])
        data['LOOKBACK'].append(ev_results['hyperparameters']['lookback'])
        data['Hidden Layer Size'].append(ev_results['hyperparameters']['hidden_layer_size'])
        data['Features Number'].append(ev_results['hyperparameters']['features_number'])
        data['T-MASE'].append(ev_results['test_error'])
        data['T-MASE Naive'].append(ev_results['naive_test_error'])
        data['V-MASE'].append(ev_results['validation_error'])
        data['V-MASE Naive'].append(ev_results['naive_validation_error'])
        
        test_naive_improv = (ev_results["naive_test_error"]- ev_results["test_error"])*100/ev_results["naive_test_error"]
        #test_snaive_improv = (ev_results["snaive_test_error"]- ev_results["test_error"])*100/ev_results["snaive_test_error"]

        validation_naive_improv = (ev_results["naive_validation_error"]- ev_results["validation_error"])*100/ev_results["naive_test_error"]
        #validation_snaive_improv = (ev_results["snaive_validation_error"]- ev_results["validation_error"])*100/ev_results["snaive_test_error"]
        
        data['T-Imporvement %'].append(round(test_naive_improv,3))
        
        #data['Test Imporvement sNaive %'].append(round(test_snaive_improv,3))
        data['V-Imporvement %'].append(round(validation_naive_improv, 3))
        #data['Validation Imporvement sNaive %'].append(round(validation_snaive_improv,3))

df = pd.DataFrame(data=data)

dfStyler = df.style.set_properties(**{'text-align': 'center'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])


Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk


,Model,Training Loss Function,Epochs,LOOKBACK,Hidden Layer Size,Features Number,T-MASE,T-MASE Naive,V-MASE,V-MASE Naive,T-Imporvement %,V-Imporvement %
0,1-LSTM,qd_objective_lstm_c,30,652,80,1,8.037,12.638,1.534,2.703,36.406,9.25
1,1-LSTM,qd_objective_lstm_c,30,48,80,1,8.408,20.57,1.932,3.689,59.125,8.542
2,1-LSTM,qd_objective_lstm_c,100,652,80,1,8.121,12.638,1.534,2.703,35.741,9.25
3,1-LSTM,qd_objective_lstm_c,30,48,40,1,15.44,20.57,2.44,3.689,24.939,6.072
4,1-LSTM,qd_objective_lstm_c,30,48,40,1,14.085,20.57,1.812,3.689,31.526,9.125
5,1-LSTM,qd_objective_lstm_c,10,48,30,1,8.623,20.57,1.778,3.689,58.08,9.29
6,1-LSTM,qd_objective_lstm_c,30,48,40,1,14.594,20.57,1.911,3.689,29.052,8.644
7,3-LSTM,qd_objective_lstm_c,100,48,200,1,14.912,20.57,1.795,3.689,27.506,9.208
8,3-LSTM,qd_objective_lstm_c,10,48,120,1,14.664,20.57,2.088,3.689,28.712,7.783
9,3-LSTM,qd_objective_lstm_c,20,48,50,1,30.924,20.57,5.33,3.689,-50.335,-7.978


In [4]:
df.to_csv('../models/berken/comparsion_point_predection.csv', sep='\t')

# Full Evaluation And Comparsion ( Intervals Prediction Test)

In [2]:
diffAugmentation = DiffAugmentation()
x_augmentations = []
y_augmentations = []

data = {'Model':[], 'Training Loss Function':[], 'Epochs':[], 'LOOKBACK': [], 'Hidden Layer Size':[],
        'Features Number':[], 'ACD': [], 'ACD Naive': [], 'MSIS': [], 'MSIS Naive': [],
        'ACD-Imporvement %': [],'MSIS-Imporvement %': [], 
       }

models_arch = ['1-LSTM', '2-LSTM', '3-LSTM']
loss_functions = [m4_mase]

MODEL_BASE_DIR = '../models/berken'
TRAINING_DATA_PATH = '../Dataset/Train/Hourly-train.csv'
TEST_DATA_PATH = '../Dataset/Test/Hourly-test.csv'

for model_arc in models_arch:
    s = f'{MODEL_BASE_DIR}/{model_arc}/*/'
    models = glob(s)
    for model_path in models:
        
        ev_results = {}
        
        for loss in loss_functions:
            ev_results = load_and_evaluate_model_PI(model_path, '../Dataset/Train/Hourly-train.csv', 
                                                 '../Dataset/Test/Hourly-test.csv', 
                                                 x_augmentations, y_augmentations, evaluate_berken_PI)
        data['Model'].append(model_arc)
        data['Training Loss Function'].append(ev_results['hyperparameters']['loss'])
        data['Epochs'].append(ev_results['hyperparameters']['epochs'])
        data['LOOKBACK'].append(ev_results['hyperparameters']['lookback'])
        data['Hidden Layer Size'].append(ev_results['hyperparameters']['hidden_layer_size'])
        data['Features Number'].append(ev_results['hyperparameters']['features_number'])
        data['ACD'].append(ev_results['acd_test'])
        data['ACD Naive'].append(ev_results['acd_naive_test'])
        data['MSIS'].append(ev_results['msis_test'])
        data['MSIS Naive'].append(ev_results['msis_naive_test'])
        
        acd_naive_improv = (ev_results["acd_naive_test"]- ev_results["acd_test"])*100/ev_results["acd_naive_test"]

        msis_naive_improv = (ev_results["msis_naive_test"]- ev_results["msis_test"])*100/ev_results["msis_naive_test"]
        
        data['ACD-Imporvement %'].append(round(acd_naive_improv,3))
        data['MSIS-Imporvement %'].append(round(msis_naive_improv, 3))

df = pd.DataFrame(data=data)

dfStyler = df.style.set_properties(**{'text-align': 'center'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])


Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk


,Model,Training Loss Function,Epochs,LOOKBACK,Hidden Layer Size,Features Number,ACD,ACD Naive,MSIS,MSIS Naive,ACD-Imporvement %,MSIS-Imporvement %
0,1-LSTM,qd_objective_lstm_c,30,652,80,1,0.026,0.046,104.658,50.34,43.478,-107.902
1,1-LSTM,qd_objective_lstm_c,30,48,80,1,0.019,0.046,160.949,81.166,58.696,-98.296
2,1-LSTM,qd_objective_lstm_c,100,652,80,1,0.039,0.046,70.048,50.34,15.217,-39.15
3,1-LSTM,qd_objective_lstm_c,30,48,40,1,0.004,0.046,171.718,81.166,91.304,-111.564
4,1-LSTM,qd_objective_lstm_c,30,48,40,1,0.017,0.046,277.697,81.166,63.043,-242.135
5,1-LSTM,qd_objective_lstm_c,10,48,30,1,0.01,0.046,74.633,81.166,78.261,8.049
6,1-LSTM,qd_objective_lstm_c,30,48,40,1,0.027,0.046,140.131,81.166,41.304,-72.647
7,3-LSTM,qd_objective_lstm_c,100,48,200,1,0.014,0.046,323.744,81.166,69.565,-298.867
8,3-LSTM,qd_objective_lstm_c,10,48,120,1,0.002,0.046,489.314,81.166,95.652,-502.856
9,3-LSTM,qd_objective_lstm_c,20,48,50,1,0.034,0.046,245.771,81.166,26.087,-202.8


In [5]:
df.to_csv('../models/berken/comparsion_pi_test.csv', sep='\t')

# Full Evaluation And Comparsion ( Intervals Prediction Validation)

In [9]:
diffAugmentation = DiffAugmentation()
x_augmentations = []
y_augmentations = []

data = {'Model':[], 'Training Loss Function':[], 'Epochs':[], 'LOOKBACK': [], 'Hidden Layer Size':[],
        'Features Number':[], 'ACD': [], 'ACD Naive': [], 'MSIS': [], 'MSIS Naive': [],
        'ACD-Imporvement %': [],'MSIS-Imporvement %': [], 
       }

models_arch = ['1-LSTM', '2-LSTM', '3-LSTM']
loss_functions = [m4_mase]

MODEL_BASE_DIR = '../models/berken'
TRAINING_DATA_PATH = '../Dataset/Train/Hourly-train.csv'
TEST_DATA_PATH = '../Dataset/Test/Hourly-test.csv'

for model_arc in models_arch:
    s = f'{MODEL_BASE_DIR}/{model_arc}/*/'
    models = glob(s)
    for model_path in models:
        
        ev_results = {}
        
        for loss in loss_functions:
            ev_results = load_and_evaluate_model_PI(model_path, '../Dataset/Train/Hourly-train.csv', 
                                                 '../Dataset/Test/Hourly-test.csv', 
                                                 x_augmentations, y_augmentations, evaluate_berken_PI)
        data['Model'].append(model_arc)
        data['Training Loss Function'].append(ev_results['hyperparameters']['loss'])
        data['Epochs'].append(ev_results['hyperparameters']['epochs'])
        data['LOOKBACK'].append(ev_results['hyperparameters']['lookback'])
        data['Hidden Layer Size'].append(ev_results['hyperparameters']['hidden_layer_size'])
        data['Features Number'].append(ev_results['hyperparameters']['features_number'])
        data['ACD'].append(ev_results['acd_validation'])
        data['ACD Naive'].append(ev_results['acd_naive_validation'])
        data['MSIS'].append(ev_results['msis_validation'])
        data['MSIS Naive'].append(ev_results['msis_naive_validation'])
        
        acd_naive_improv = (ev_results["acd_naive_validation"]- ev_results["acd_validation"])*100/ev_results["acd_naive_validation"]

        msis_naive_improv = (ev_results["msis_naive_validation"]- ev_results["msis_validation"])*100/ev_results["msis_naive_validation"]
        
        data['ACD-Imporvement %'].append(round(acd_naive_improv,3))
        data['MSIS-Imporvement %'].append(round(msis_naive_improv, 3))

df = pd.DataFrame(data=data)

dfStyler = df.style.set_properties(**{'text-align': 'center'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])


Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk


,Model,Training Loss Function,Epochs,LOOKBACK,Hidden Layer Size,Features Number,ACD,ACD Naive,MSIS,MSIS Naive,ACD-Imporvement %,MSIS-Imporvement %
0,1-LSTM,qd_objective_lstm_c,30,652,80,1,0.023,0.04,21.128,9.761,42.5,-116.453
1,1-LSTM,qd_objective_lstm_c,30,48,80,1,0.016,0.039,25.761,14.923,58.974,-72.626
2,1-LSTM,qd_objective_lstm_c,100,652,80,1,0.033,0.04,14.55,9.761,17.5,-49.063
3,1-LSTM,qd_objective_lstm_c,30,48,40,1,0.024,0.039,30.945,14.923,38.462,-107.364
4,1-LSTM,qd_objective_lstm_c,30,48,40,1,0.017,0.039,46.209,14.923,56.41,-209.65
5,1-LSTM,qd_objective_lstm_c,10,48,30,1,0.007,0.039,15.865,14.923,82.051,-6.312
6,1-LSTM,qd_objective_lstm_c,30,48,40,1,0.044,0.039,28.464,14.923,-12.821,-90.739
7,3-LSTM,qd_objective_lstm_c,100,48,200,1,0.013,0.039,52.973,14.923,66.667,-254.976
8,3-LSTM,qd_objective_lstm_c,10,48,120,1,0.003,0.039,85.948,14.923,92.308,-475.943
9,3-LSTM,qd_objective_lstm_c,20,48,50,1,0.015,0.039,41.596,14.923,61.538,-178.738


In [10]:
df.to_csv('../models/berken/comparsion_pi_validation.csv', sep='\t')